# Creator 🏗
### Create Django projects and their respective applications.

In [1]:
! D create --help

Usage: D create [OPTIONS] COMMAND [ARGS]...

  Creates projects and apps.

Options:
  --dry      Display output without creating files.
  --default  Apply all default configurations.
  --verbose  Run in verbose mode.
  --help     Show this message and exit.

Commands:
  app       Creates new django apps.
  project   Creates a new django project.
  settings  Create settings file for project.


The `create` command (abbreviated `c`) can be used to start new projects as well as new applications. The command tries to simplify how a project is created as well as the applications contained in it. Here's an example of such simplification:

Suppose you want to start a new project and want to create two apps within it:

```

```

```
django-admin startproject website && \
cd website/website/ && \
django-admin startapp blog && \
django-admin startapp music
```

The equivalent command in the `django-clite` is:

In [61]:
! D create project website blog music

Attempting to create project: website
Successfully created application: blog
Successfully created application: music
Successfully initialized git repository


Specifying `apps` when creating a project is optional, but you're likely to need to create one inside of your project directory, so the CLI can handle the creation of all of your apps if you pass them as arguments after your project name.

## Creating new projects
To create a new project, simply run `D create project project_name`. You can also specify the following flags:

In [62]:
! D create project --help

Usage: D create project [OPTIONS] NAME [APPS]...

  Creates a new django project.

  This is similar to using `django-admin startproject project_name` but with
  some added functionality. The command can handle the creation of apps upon
  creation of any project, you can simply specify the name of the apps after
  the project name:

      D new project website app1 app2 app3 ...

  Another added functionality is the ability to setup Docker and Dokku. To
  do so, use the respective flags --docker and --dokku.

  Also helpful is the ability to initialize your application with a custom
  authentication model of your choosing. When the --custom-auth flag is
  specified, CLI will add an `authentication` app to your project along with
  a subclass of the AbstractUser model which you can extend for your own
  authentication purposes.

Options:
  --docker           Add support for Docker.
  --dokku            Add support for Dokku.
  -a, --custom-auth  Add support for custom AUTH_USER_MODEL.
 

The `--custom-auth` flag is used to provide a simple override of the `AUTH_USER_MODEL`. This creates a User model under authentication.models.user. One can simply specify the override in `settings.py` by setting:

```
AUTH_USER_MODEL = 'authentication.User'
```

The `--docker` flag will create a `Dockerfile` as well as a `docker-compose.yml` file within your project. These are pre-configured to support the following services: web (the Django application itself), a database (postgres), and a caching server (redis).

### Project structure
This CLI makes some assumptions about the structure of your Django project.

1. It assumes that your apps are one level below the root of your project directory, one level below where `manage.py` is. For example, here's a project generated with defaults:

In [63]:
! tree --dirsfirst -L 3 website

website
├── website
│   ├── blog
│   │   ├── admin
│   │   ├── fixtures
│   │   ├── forms
│   │   ├── middleware
│   │   ├── migrations
│   │   ├── models
│   │   ├── serializers
│   │   ├── templates
│   │   ├── views
│   │   ├── viewsets
│   │   ├── apps.py
│   │   └── urls.py
│   ├── music
│   │   ├── admin
│   │   ├── fixtures
│   │   ├── forms
│   │   ├── middleware
│   │   ├── migrations
│   │   ├── models
│   │   ├── serializers
│   │   ├── templates
│   │   ├── views
│   │   ├── viewsets
│   │   ├── apps.py
│   │   └── urls.py
│   ├── __init__.py
│   ├── asgi.py
│   ├── settings.py
│   ├── settings_override.py
│   ├── storage.py
│   ├── urls.py
│   └── wsgi.py
└── manage.py

23 directories, 12 files


2. It assumes that your app resources are grouped together by type in packages. For example:

In [64]:
! tree --dirsfirst -L 1 website/website/music

website/website/music
├── admin
├── fixtures
├── forms
├── middleware
├── migrations
├── models
├── serializers
├── templates
├── views
├── viewsets
├── apps.py
└── urls.py

10 directories, 2 files


3. Each class representing a `model`, `serializer`, `viewset`, or form is located in its own Python module. For example:

In [105]:
! tree -L 1 -P '*.py' --prune website/website/music/models

website/website/music/models
├── __init__.py
└── album.py

0 directories, 2 files


This is done in order to aid the CLI with the creation and deletion of files
in the project as we'll see under the [`generate`](#generator) and [`destroy`](#destroyer) commands.

## Creating apps

In [4]:
! D create app --help

Usage: D create app [OPTIONS] [APPS]...

  Creates new django apps.

  This is similar to using `django-admin startapp app_name` but adds more
  structure to your app by creating packages for forms, models, serializers,
  tests, templates, views, and viewsets.

  The command can accept multiple apps as arguments. So,

      D new app shop blog forum

  will work prompt the CLI to create all 4 apps two levels within your
  project's directory, i.e mysite/mysite. The CLI tries to identify where
  the management module for your project is so it can place your app files
  in the correct location. This helps with consistency as the CLI can infer
  module/package scopes when performing other automatic configurations.

  As part of the CLI convention, each app is assigned its own `urls.py`
  file, which can be used to route urls on a per app basis. Another
  convention the CLI adopts is to add a viewsets package to the app's
  directory by default (for use with DRF). Within the viewsets direc